<a href="https://colab.research.google.com/github/ottavio-c/test_data_science/blob/main/Dashboard_Packaging_Volume.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Environment Setting**

In [1]:
!pip install pandas plotly ipywidgets

In [2]:
from google.colab import output
output.enable_custom_widget_manager()

import pandas as pd
import numpy as np
import random

import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, HTML

In [3]:
# Set seed for reproducibility
np.random.seed(42)
random.seed(42)

**Dummy Data Generation**

In [4]:
# Number of unique rows you want
n_rows = 200

# Define possible values for each field
manufacturing_plants = ['Turin', 'Milan', 'Berlin', 'Madrid', 'Paris']
manufacturers = ['OMNIAUTO', 'AUTOTECH', 'PARTSOLVE']
model_codes = [str(code) for code in range(7000, 8000)]
merch_groups = ['INTERIOR', 'EXTERIOR', 'ENGINE', 'ELECTRICAL']
countries_iso = ['FR', 'ES', 'DE', 'IT']
countries_name = {'FR': 'France', 'ES': 'Spain', 'DE': 'Germany', 'IT': 'Italy'}
cities = {
    'France': ['Toulouse', 'Lyon', 'Paris'],
    'Spain': ['Barcelona', 'Valencia', 'Madrid'],
    'Germany': ['Wolfsburg', 'Berlin', 'Munich'],
    'Italy': ['Turin', 'Milan', 'Rome']
}
incoterms = ['CIP', 'DDP', 'FCA', 'DAP', 'CPT']
reusable_flags = ['P', 'D']  # P = reusable, D = disposable

# Generate coordinates for cities
def random_coords(city_list):
    return {
        city: (
            round(np.random.uniform(40, 55), 6),  # Latitude
            round(np.random.uniform(-5, 15), 6)   # Longitude
        )
        for city in city_list
    }

city_coords = {city: coord for cities_list in cities.values() for city, coord in random_coords(cities_list).items()}

# Use a set to ensure rows are unique
unique_rows = set()
data = []

while len(data) < n_rows:
    plant = random.choice(manufacturing_plants)
    manufacturer = random.choice(manufacturers)
    model_code = random.choice(model_codes)
    merch_group = random.choice(merch_groups)
    country_iso = random.choice(countries_iso)
    country_name = countries_name[country_iso]
    city = random.choice(cities[country_name])
    incoterm = random.choice(incoterms)
    reusable = random.choice(reusable_flags)
    kpi = round(np.random.uniform(0.002, 0.05), 6)
    lat, lon = city_coords[city]

    # Ensure uniqueness based on a composite key
    key = (plant, manufacturer, model_code, city, kpi)
    if key not in unique_rows:
        unique_rows.add(key)
        data.append([
            plant,
            manufacturer,
            model_code,
            merch_group,
            country_iso,
            country_name,
            city,
            incoterm,
            reusable,
            kpi,
            lat,
            lon
        ])

# Define column names (matching your original file)
columns = [
    "GN_IN_MANUFACTURING_PLANT_NAME",
    "GN_IN_MANUFACTURER_NAME",
    "CD_IN_MODEL_CODE",
    "GN_IN_MERCHANDISING_GROUP_NAME",
    "CD_IN_SHIPPER_COUNTRY_ISO_CODE",
    "GN_IN_SHIPPER_COUNTRY_ISO_NAME",
    "GN_IN_SHIPPER_CITY_NAME",
    "CD_IN_INTERNATIONAL_COMMERCIAL_TERMS_CODE",
    "FL_IN_IS_PACKAGING_REUSABLE_FLAG",
    "NM_IN_KPI_PACKAGING_VOLUME_PER_PART_USAGE_RATIO_NUMBER",
    "GN_IN_SHIPPER_CITY_LATITUDE",
    "GN_IN_SHIPPER_CITY_LONGITUDE"
]

# Create the DataFrame
df = pd.DataFrame(data, columns=columns)

# Show the first few rows
df.head(5)

,GN_IN_MANUFACTURING_PLANT_NAME,GN_IN_MANUFACTURER_NAME,CD_IN_MODEL_CODE,GN_IN_MERCHANDISING_GROUP_NAME,CD_IN_SHIPPER_COUNTRY_ISO_CODE,GN_IN_SHIPPER_COUNTRY_ISO_NAME,GN_IN_SHIPPER_CITY_NAME,CD_IN_INTERNATIONAL_COMMERCIAL_TERMS_CODE,FL_IN_IS_PACKAGING_REUSABLE_FLAG,NM_IN_KPI_PACKAGING_VOLUME_PER_PART_USAGE_RATIO_NUMBER,GN_IN_SHIPPER_CITY_LATITUDE,GN_IN_SHIPPER_CITY_LONGITUDE
0,Turin,OMNIAUTO,7759,ENGINE,ES,Spain,Barcelona,DDP,P,0.023891,40.871254,12.323523
1,Paris,OMNIAUTO,7604,ELECTRICAL,FR,France,Toulouse,CIP,P,0.039688,45.618102,14.014286
2,Milan,PARTSOLVE,7616,INTERIOR,ES,Spain,Madrid,CPT,D,0.011584,40.308767,14.398197
3,Milan,AUTOTECH,7603,ENGINE,FR,France,Toulouse,DAP,D,0.026683,45.618102,14.014286
4,Berlin,OMNIAUTO,7220,ENGINE,FR,France,Toulouse,DAP,P,0.030436,45.618102,14.014286


**Add ID column to be used as key**



In [5]:
# Create an 'ID' field to serve as the primary key
df["ID"] = df.index.astype(str)

# Reorder columns to have 'ID' as the first column
df = df[["ID"] + [col for col in df.columns if col != "ID"]]

In [6]:
# Display the first few rows to verify
df.head(5)

,ID,GN_IN_MANUFACTURING_PLANT_NAME,GN_IN_MANUFACTURER_NAME,CD_IN_MODEL_CODE,GN_IN_MERCHANDISING_GROUP_NAME,CD_IN_SHIPPER_COUNTRY_ISO_CODE,GN_IN_SHIPPER_COUNTRY_ISO_NAME,GN_IN_SHIPPER_CITY_NAME,CD_IN_INTERNATIONAL_COMMERCIAL_TERMS_CODE,FL_IN_IS_PACKAGING_REUSABLE_FLAG,NM_IN_KPI_PACKAGING_VOLUME_PER_PART_USAGE_RATIO_NUMBER,GN_IN_SHIPPER_CITY_LATITUDE,GN_IN_SHIPPER_CITY_LONGITUDE
0,0,Turin,OMNIAUTO,7759,ENGINE,ES,Spain,Barcelona,DDP,P,0.023891,40.871254,12.323523
1,1,Paris,OMNIAUTO,7604,ELECTRICAL,FR,France,Toulouse,CIP,P,0.039688,45.618102,14.014286
2,2,Milan,PARTSOLVE,7616,INTERIOR,ES,Spain,Madrid,CPT,D,0.011584,40.308767,14.398197
3,3,Milan,AUTOTECH,7603,ENGINE,FR,France,Toulouse,DAP,D,0.026683,45.618102,14.014286
4,4,Berlin,OMNIAUTO,7220,ENGINE,FR,France,Toulouse,DAP,P,0.030436,45.618102,14.014286


**Simulate the Foundry Object Type (Ontology) mapping**

---



In [7]:
# Rename columns to simulate Foundry Object Type mapping
df = df.rename(columns={
    "ID": "id",  # Primary Key
    "GN_IN_MANUFACTURING_PLANT_NAME": "manufacturingPlant",
    "GN_IN_MANUFACTURER_NAME": "supplier",
    "CD_IN_MODEL_CODE": "modelCode",
    "GN_IN_MERCHANDISING_GROUP_NAME": "merchandisingGroup",
    "CD_IN_SHIPPER_COUNTRY_ISO_CODE": "shipperCountryCode",
    "GN_IN_SHIPPER_COUNTRY_ISO_NAME": "shipperCountryName",
    "GN_IN_SHIPPER_CITY_NAME": "shipperCity",
    "GN_IN_SHIPPER_CITY_LATITUDE": "latitude",
    "GN_IN_SHIPPER_CITY_LONGITUDE": "longitude",
    "CD_IN_INTERNATIONAL_COMMERCIAL_TERMS_CODE": "incoterm",
    "FL_IN_IS_PACKAGING_REUSABLE_FLAG": "isReusable",
    "NM_IN_KPI_PACKAGING_VOLUME_PER_PART_USAGE_RATIO_NUMBER": "kpiVolumeRatio"
})

# Display the first few rows to verify
df.head(5)

,id,manufacturingPlant,supplier,modelCode,merchandisingGroup,shipperCountryCode,shipperCountryName,shipperCity,incoterm,isReusable,kpiVolumeRatio,latitude,longitude
0,0,Turin,OMNIAUTO,7759,ENGINE,ES,Spain,Barcelona,DDP,P,0.023891,40.871254,12.323523
1,1,Paris,OMNIAUTO,7604,ELECTRICAL,FR,France,Toulouse,CIP,P,0.039688,45.618102,14.014286
2,2,Milan,PARTSOLVE,7616,INTERIOR,ES,Spain,Madrid,CPT,D,0.011584,40.308767,14.398197
3,3,Milan,AUTOTECH,7603,ENGINE,FR,France,Toulouse,DAP,D,0.026683,45.618102,14.014286
4,4,Berlin,OMNIAUTO,7220,ENGINE,FR,France,Toulouse,DAP,P,0.030436,45.618102,14.014286


**Dashboard Definition**

In [8]:
# Interactive filters (default: all selected)
plant_filter = widgets.SelectMultiple(
    options=sorted(df['manufacturingPlant'].dropna().unique()),
    value=tuple(df['manufacturingPlant'].dropna().unique()),
    description='Plant:',
    layout=widgets.Layout(width='180px')
)

model_filter = widgets.SelectMultiple(
    options=sorted(df['modelCode'].dropna().unique()),
    value=tuple(df['modelCode'].dropna().unique()),
    description='Model:',
    layout=widgets.Layout(width='180px')
)

country_filter = widgets.SelectMultiple(
    options=sorted(df['shipperCountryName'].dropna().unique()),
    value=tuple(df['shipperCountryName'].dropna().unique()),
    description='Country:',
    layout=widgets.Layout(width='180px')
)

incoterm_filter = widgets.SelectMultiple(
    options=sorted(df['incoterm'].dropna().unique()),
    value=tuple(df['incoterm'].dropna().unique()),
    description='Incoterm:',
    layout=widgets.Layout(width='180px')
)

reusable_filter = widgets.SelectMultiple(
    options=sorted(df['isReusable'].dropna().unique()),
    value=tuple(df['isReusable'].dropna().unique()),
    description='Reusable:',
    layout=widgets.Layout(width='180px')
)

In [14]:
# Dashboard update function
def update_dashboard(plant, model, country, incoterm, reusable):
    dff = df.copy()

    # Apply filters
    if plant:
        dff = dff[dff['manufacturingPlant'].isin(plant)]
    if model:
        dff = dff[dff['modelCode'].isin(model)]
    if country:
        dff = dff[dff['shipperCountryName'].isin(country)]
    if incoterm:
        dff = dff[dff['incoterm'].isin(incoterm)]
    if reusable:
        dff = dff[dff['isReusable'].isin(reusable)]

    # Show KPI box
    total_kpi = dff['kpiVolumeRatio'].sum()
    display(HTML(f"""
        <div style='
            padding: 10px;
            background-color: #f0f0f0;
            border: 1px solid #ccc;
            border-radius: 6px;
            font-weight: bold;
            font-size: 18px;
            margin-bottom: 15px;
        '>
            Packaging Volume (Total): {total_kpi:.4f}
        </div>
    """))

    # Show map immediately after KPI
    fig_geo = px.scatter_geo(
        dff,
        lat="latitude",
        lon="longitude",
        size="kpiVolumeRatio",
        color="manufacturingPlant",
        hover_name="modelCode",
        projection="natural earth",
        title="Geographic Distribution of Packaging Volume"
    )
    fig_geo.show()

    # Handle empty result
    if dff.empty:
        display(HTML(f"""
            <div style='
                padding: 10px;
                background-color: #ffe6e6;
                border: 1px solid #cc0000;
                border-radius: 6px;
                font-weight: bold;
                color: #cc0000;
                font-size: 16px;
                margin-top: 15px;
            '>
                No data matches the selected filters.
            </div>
        """))
        return

    # Treemap by merchandising group
    fig_merch = px.treemap(
        dff,
        path=['merchandisingGroup'],
        values='kpiVolumeRatio',
        title='Packaging Volume by Merchandising Group'
    )
    fig_merch.show()

    # Treemap by supplier
    fig_supplier = px.treemap(
        dff,
        path=['supplier'],
        values='kpiVolumeRatio',
        title='Packaging Volume by Supplier'
    )
    fig_supplier.show()


In [31]:
# Filter User Interface and Interactive Output

# Combine all filter widgets horizontally
filters_box = widgets.HBox([
    plant_filter,
    model_filter,
    country_filter,
    incoterm_filter,
    reusable_filter
])

# Link filter values to the update_dashboard function
out = widgets.interactive_output(update_dashboard, {
    'plant': plant_filter,
    'model': model_filter,
    'country': country_filter,
    'incoterm': incoterm_filter,
    'reusable': reusable_filter
})

**Display the Dashboard**

In [32]:
# Optional: reset output layout to default (no scroll, full expansion)
out.layout = widgets.Layout()  # no height/overflow set

# Combine filters and output in a vertical layout
dashboard_layout = widgets.VBox([
    filters_box,
    out  # expands naturally
])

# Display entire dashboard (everything shown by default)
display(dashboard_layout)